---
**Turma:** 15

**Grupo:** 11 - Bruno Correia, Gilson Costa, Vivian Liu

**Contato**: `brunovpm@hotmail.com, gilson.costa@gmail.com, liuoliveira.vivian@gmail.com`

**Licença deste notebook**:
![CC BY](https://licensebuttons.net/l/by/3.0/88x31.png)

[Clique aqui para saber mais sobre a licença CC BY v4.0](https://creativecommons.org/licenses/by/4.0/legalcode.pt)

# ETL de dados de push e session
- Geração de dados agrupados por dia
- Geração de dados agrupados por mês
- Geração de dados agrupados por dia da semana

Utilize o comando do `dbutils.fs.rm` para remover arquivos e diretórios dentro do DBFS.

In [0]:
# dbutils.fs.rm('/dbfs/FileStore/ifood/orders', True)

In [0]:
root_dir = '/dbfs/FileStore/ifood'
dbutils.fs.ls(f'{root_dir}')

Out[1]: [FileInfo(path='dbfs:/dbfs/FileStore/ifood/abt/', name='abt/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/customer_segmentation/', name='customer_segmentation/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/enrich/', name='enrich/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/marketing_push_full/', name='marketing_push_full/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/orders/', name='orders/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/orders_with_cost_revenue/', name='orders_with_cost_revenue/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/preprocess/', name='preprocess/', size=0),
 FileInfo(path='dbfs:/dbfs/FileStore/ifood/sessions_visits/', name='sessions_visits/', size=0)]

In [0]:
df_marketing_push_full = spark.read.parquet(f'{root_dir}/marketing_push_full')
print((df_marketing_push_full.count(), len(df_marketing_push_full.columns)))

#df_sessions_visits = spark.read.parquet(f'{root_dir}/sessions_visits')
#print((df_sessions_visits.count(), len(df_sessions_visits.columns)))

(28426642, 21)

In [0]:
from pyspark.sql.functions import isnull, when, count, col
import pandas as pd

pd.options.display.precision = 4

def summaryZeroNull(df, lista):
  df_null = df.select([count(when(isnull(c), 1)).alias(c) for c in lista])
  df_zero = df.select([count(when(df[c] == 0, 1)).alias(c) for c in lista])

  final = df_null.union(df_zero)

  dfPandas = final.toPandas()
  
  dfTranspose = dfPandas.transpose(copy=True)
  dfTranspose.reset_index(inplace=True);
  dfTranspose.rename(columns={'index':'summary', 0:'null',1:'zero'}, inplace=True)
  dfTranspose
  
  return dfTranspose

# marketing_push_full

_26 % de registros duplicados_

In [0]:
# eliminando colunas preenchidas sempre com o mesmo valor
df_marketing_unique = df_marketing_push_full.drop('brand', 'event_channel', 'user_id')
# eliminando registros duplicado
df_marketing_unique = df_marketing_unique.drop_duplicates()
df_marketing_unique.write.parquet(f'{root_dir}/preprocess/push', mode='overwrite')

In [0]:
df_marketing_push_full = spark.read.parquet(f'{root_dir}/preprocess/push')
print((df_marketing_push_full.count(), len(df_marketing_push_full.columns)))

(20875367, 18)

In [0]:
df_marketing_push_full.createOrReplaceTempView("push")

# Sumário diário de eventos do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW push_user_prepare AS
SELECT
event_date,
dayofweek(event_date) event_dow,
hour(event_time_utc3) event_hour,
external_user_id,
event_name

FROM push

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW push_user_day_view AS
SELECT
event_date,
event_dow,
external_user_id,

SUM(IF(event_hour BETWEEN 0 AND 4, 1, 0)) total_event_dawn,
SUM(IF(event_hour BETWEEN 5 AND 9, 1, 0)) total_event_breakfast,
SUM(IF(event_hour BETWEEN 10 AND 14, 1, 0)) total_event_lunch,
SUM(IF(event_hour BETWEEN 15 AND 16, 1, 0)) total_event_snack,
SUM(IF(event_hour BETWEEN 17 AND 23, 1, 0)) total_event_dinner,

SUM(IF(event_name = 'send' AND event_hour BETWEEN 0 AND 4, 1, 0)) total_send_dawn,
SUM(IF(event_name = 'send' AND event_hour BETWEEN 5 AND 9, 1, 0)) total_send_breakfast,
SUM(IF(event_name = 'send' AND event_hour BETWEEN 10 AND 14, 1, 0)) total_send_lunch,
SUM(IF(event_name = 'send' AND event_hour BETWEEN 15 AND 16, 1, 0)) total_send_snack,
SUM(IF(event_name = 'send' AND event_hour BETWEEN 17 AND 23, 1, 0)) total_send_dinner,

SUM(IF(event_name = 'received' AND event_hour BETWEEN 0 AND 4, 1, 0)) total_received_dawn,
SUM(IF(event_name = 'received' AND event_hour BETWEEN 5 AND 9, 1, 0)) total_received_breakfast,
SUM(IF(event_name = 'received' AND event_hour BETWEEN 10 AND 14, 1, 0)) total_received_lunch,
SUM(IF(event_name = 'received' AND event_hour BETWEEN 15 AND 16, 1, 0)) total_received_snack,
SUM(IF(event_name = 'received' AND event_hour BETWEEN 17 AND 23, 1, 0)) total_received_dinner,

SUM(IF(event_name = 'bounce' AND event_hour BETWEEN 0 AND 4, 1, 0)) total_bounce_dawn,
SUM(IF(event_name = 'bounce' AND event_hour BETWEEN 5 AND 9, 1, 0)) total_bounce_breakfast,
SUM(IF(event_name = 'bounce' AND event_hour BETWEEN 10 AND 14, 1, 0)) total_bounce_lunch,
SUM(IF(event_name = 'bounce' AND event_hour BETWEEN 15 AND 16, 1, 0)) total_bounce_snack,
SUM(IF(event_name = 'bounce' AND event_hour BETWEEN 17 AND 23, 1, 0)) total_bounce_dinner,

SUM(IF(event_name = 'click' AND event_hour BETWEEN 0 AND 4, 1, 0)) total_click_dawn,
SUM(IF(event_name = 'click' AND event_hour BETWEEN 5 AND 9, 1, 0)) total_click_breakfast,
SUM(IF(event_name = 'click' AND event_hour BETWEEN 10 AND 14, 1, 0)) total_click_lunch,
SUM(IF(event_name = 'click' AND event_hour BETWEEN 15 AND 16, 1, 0)) total_click_snack,
SUM(IF(event_name = 'click' AND event_hour BETWEEN 17 AND 23, 1, 0)) total_click_dinner,

count(*) total_event,
SUM(IF(event_name = 'send', 1, 0)) total_send,
SUM(IF(event_name = 'received', 1, 0)) total_received,
SUM(IF(event_name = 'bounce', 1, 0)) total_bounce,
SUM(IF(event_name = 'click', 1, 0)) total_click

FROM push_user_prepare

GROUP BY 1, 2, 3

In [0]:
df_push_user_day_view = spark.table('push_user_day_view')
df_push_user_day_view.write.parquet(f'{root_dir}/enrich/push_user_day', mode='overwrite')

In [0]:
df_push_user_day = spark.read.parquet(f'{root_dir}/enrich/push_user_day')
print((df_push_user_day.count(), len(df_push_user_day.columns)))

(3891492, 33)

In [0]:
df_push_user_day.createOrReplaceTempView("push_user_day")

In [0]:
%sql
SELECT *
FROM push_user_day
WHERE external_user_id = 'c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a'

event_date,event_dow,external_user_id,total_event_dawn,total_event_breakfast,total_event_lunch,total_event_snack,total_event_dinner,total_send_dawn,total_send_breakfast,total_send_lunch,total_send_snack,total_send_dinner,total_received_dawn,total_received_breakfast,total_received_lunch,total_received_snack,total_received_dinner,total_bounce_dawn,total_bounce_breakfast,total_bounce_lunch,total_bounce_snack,total_bounce_dinner,total_click_dawn,total_click_breakfast,total_click_lunch,total_click_snack,total_click_dinner,total_event,total_send,total_received,total_bounce,total_click
2019-06-03T00:00:00.000Z,2,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,2,0,2,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,4,2,2,0,0
2019-06-21T00:00:00.000Z,6,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,3,0,4,0,0,1,0,2,0,0,1,0,2,0,0,0,0,0,0,0,1,0,0,7,3,3,0,1
2019-06-24T00:00:00.000Z,2,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,3,3,3,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,9,3,3,0,3
2019-06-28T00:00:00.000Z,6,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,0,0,3,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,3,1,1,0,1
2019-06-04T00:00:00.000Z,3,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,2,3,4,0,0,1,1,2,0,0,1,1,2,0,0,0,0,0,0,0,0,1,0,9,4,4,0,1
2019-07-16T00:00:00.000Z,3,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,2,0,3,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,5,2,2,0,1
2019-06-18T00:00:00.000Z,3,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,3,3,3,0,0,1,1,1,0,0,1,1,1,0,0,0,0,0,0,0,1,1,1,9,3,3,0,3
2019-07-22T00:00:00.000Z,2,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,1,0,1,0
2019-07-15T00:00:00.000Z,2,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,0,4,0,3,0,0,2,0,1,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,7,3,2,0,2
2019-07-12T00:00:00.000Z,6,c1836d77803e3a29efbfdab7038b49e260dffb2d5353157201604d121d80f64a,0,2,3,0,1,0,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,6,3,2,0,1


# Sumário mensal de eventos do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW push_user_month_view AS

SELECT
date_format(event_date, 'yyyy-MM-01') event_date,
external_user_id,

MAX(IF(total_send > 0, event_date, null)) last_send_date,
MAX(IF(total_received > 0, event_date, null)) last_received_date,
MAX(IF(total_bounce > 0, event_date, null)) last_bounce_date,
MAX(IF(total_click > 0, event_date, null)) last_click_date,

SUM(total_event_dawn) total_event_dawn,
SUM(total_event_breakfast) total_event_breakfast,
SUM(total_event_lunch) total_event_lunch,
SUM(total_event_snack) total_event_snack,
SUM(total_event_dinner) total_event_dinner,

SUM(total_send_dawn) total_send_dawn,
SUM(total_send_breakfast) total_send_breakfast,
SUM(total_send_lunch) total_send_lunch,
SUM(total_send_snack) total_send_snack,
SUM(total_send_dinner) total_send_dinner,

SUM(total_received_dawn) total_received_dawn,
SUM(total_received_breakfast) total_received_breakfast,
SUM(total_received_lunch) total_received_lunch,
SUM(total_received_snack) total_received_snack,
SUM(total_received_dinner) total_received_dinner,

SUM(total_bounce_dawn) total_bounce_dawn,
SUM(total_bounce_breakfast) total_bounce_breakfast,
SUM(total_bounce_lunch) total_bounce_lunch,
SUM(total_bounce_snack) total_bounce_snack,
SUM(total_bounce_dinner) total_bounce_dinner,

SUM(total_click_dawn) total_click_dawn,
SUM(total_click_breakfast) total_click_breakfast,
SUM(total_click_lunch) total_click_lunch,
SUM(total_click_snack) total_click_snack,
SUM(total_click_dinner) total_click_dinner,

SUM(total_event) total_event,
SUM(total_send) total_send,
SUM(total_received) total_received,
SUM(total_bounce) total_bounce,
SUM(total_click) total_click

FROM push_user_day

GROUP BY 1, 2

In [0]:
df_push_user_month_view = spark.table('push_user_month_view')
df_push_user_month_view.write.parquet(f'{root_dir}/enrich/push_user_month', mode='overwrite')

In [0]:
df_push_user_month = spark.read.parquet(f'{root_dir}/enrich/push_user_month').repartition(2).cache()
print((df_push_user_month.count(), len(df_push_user_month.columns)))
df_push_user_month.createOrReplaceTempView("push_user_month")

(174919, 36)

In [0]:
%sql
SELECT date_format(event_date, 'MM-MMM') event_date, sum(total_click)

FROM push_user_month

GROUP BY event_date

ORDER BY event_date

event_date,sum(total_click)
06-Jun,51497
07-Jul,55252
08-Aug,28830
09-Sep,33285
10-Oct,42071
11-Nov,45879
12-Dec,65815


In [0]:
%sql
SELECT external_user_id, COUNT(*) total, sum(total_click) total_click

FROM push_user_month

GROUP BY external_user_id

ORDER BY total DESC LIMIT 10

external_user_id,total,total_click
3fbaf376e0a4a0602cbed1b748ca4fa1159d2c29abd07d44548659af6a8f1be0,7,11
52dcc4d16f64f291ca5b3aa501fe076d5eccb3e9983929d8ef1a5b5dd3ac3fa7,7,11
c7aabe7995b2afb37ab5e3006f95bd0d7bc13274e560365dcdcb928ea455dedf,7,19
1b664ee2b0303eba7b28645f227ccc7293abb81517539c800143092e8d1ca65e,7,8
26149082563f0b769957e0cbd1f4ca8cf65afe3ba9a225836d2cbc7c157717b6,7,0
893776a1c2c97ecb025bcfcb80335429cb570a369c2b8c7412420c87365fd54a,7,12
5bd139de337fff5b59f7309e3a711a5ed63558ba6971c27b859c9aeb538959c4,7,12
777d903179c7687000113c9ca82465d75b90e0474b021ad8a3a500185f48346e,7,4
c4bcca2c4ad45b457136159237f4ec56e294994154b40e0453ad3e517fd4fa8b,7,7
bfdef00f22a82566d8e598ff70e29c59c8f80ea9f3af81b237e9b19a11cf0b5c,7,13


In [0]:
%sql
SELECT date_format(event_date, 'MM-MMM') month, *

FROM push_user_month

WHERE external_user_id = 'cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4'

ORDER BY event_date

month,event_date,external_user_id,last_send_date,last_received_date,last_bounce_date,last_click_date,total_event_dawn,total_event_breakfast,total_event_lunch,total_event_snack,total_event_dinner,total_send_dawn,total_send_breakfast,total_send_lunch,total_send_snack,total_send_dinner,total_received_dawn,total_received_breakfast,total_received_lunch,total_received_snack,total_received_dinner,total_bounce_dawn,total_bounce_breakfast,total_bounce_lunch,total_bounce_snack,total_bounce_dinner,total_click_dawn,total_click_breakfast,total_click_lunch,total_click_snack,total_click_dinner,total_event,total_send,total_received,total_bounce,total_click
06-Jun,2019-06-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-06-20T00:00:00.000Z,2019-06-19T00:00:00.000Z,2019-06-20T00:00:00.000Z,2019-06-20T00:00:00.000Z,0,0,15,2,12,0,0,7,1,6,0,0,5,1,4,0,0,2,0,2,0,0,1,0,0,29,14,10,4,1
07-Jul,2019-07-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-07-29T00:00:00.000Z,2019-07-28T00:00:00.000Z,2019-07-29T00:00:00.000Z,null,0,0,12,2,22,0,0,6,1,11,0,0,0,1,10,0,0,6,0,1,0,0,0,0,0,36,18,11,7,0
08-Aug,2019-08-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-08-26T00:00:00.000Z,2019-08-26T00:00:00.000Z,2019-08-26T00:00:00.000Z,2019-08-02T00:00:00.000Z,6,0,17,2,16,3,0,8,1,8,3,0,4,1,7,0,0,4,0,1,0,0,1,0,0,41,20,15,5,1
09-Sep,2019-09-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-09-30T00:00:00.000Z,2019-09-30T00:00:00.000Z,2019-09-30T00:00:00.000Z,2019-09-29T00:00:00.000Z,0,0,15,0,20,0,0,7,0,9,0,0,2,0,8,0,0,5,0,1,0,0,1,0,2,35,16,10,6,3
10-Oct,2019-10-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-10-29T00:00:00.000Z,2019-10-28T00:00:00.000Z,2019-10-29T00:00:00.000Z,2019-10-29T00:00:00.000Z,0,0,26,3,56,0,0,12,1,23,0,0,5,1,23,0,0,7,0,0,0,0,2,1,10,85,36,29,7,13
11-Nov,2019-11-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-11-30T00:00:00.000Z,2019-11-30T00:00:00.000Z,2019-11-28T00:00:00.000Z,2019-11-30T00:00:00.000Z,0,0,58,25,122,0,0,25,11,49,0,0,21,11,48,0,0,4,0,1,0,0,8,3,24,205,85,80,5,35
12-Dec,2019-12-01,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,2019-12-31T00:00:00.000Z,2019-12-31T00:00:00.000Z,2019-12-30T00:00:00.000Z,2019-12-31T00:00:00.000Z,0,0,60,41,143,0,0,26,17,63,0,0,24,16,62,0,0,2,1,1,0,0,8,7,17,244,106,102,4,32


In [0]:
%sql
SELECT DISTINCT event_date

FROM push_user_month

order by 1 DESC

event_date
2019-12-01
2019-11-01
2019-10-01
2019-09-01
2019-08-01
2019-07-01
2019-06-01


# Sumário semanal de eventos do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW push_user_dow_view AS

SELECT
event_dow,
external_user_id,

SUM(total_event_dawn) total_event_dawn,
SUM(total_event_breakfast) total_event_breakfast,
SUM(total_event_lunch) total_event_lunch,
SUM(total_event_snack) total_event_snack,
SUM(total_event_dinner) total_event_dinner,

SUM(total_send_dawn) total_send_dawn,
SUM(total_send_breakfast) total_send_breakfast,
SUM(total_send_lunch) total_send_lunch,
SUM(total_send_snack) total_send_snack,
SUM(total_send_dinner) total_send_dinner,

SUM(total_received_dawn) total_received_dawn,
SUM(total_received_breakfast) total_received_breakfast,
SUM(total_received_lunch) total_received_lunch,
SUM(total_received_snack) total_received_snack,
SUM(total_received_dinner) total_received_dinner,

SUM(total_bounce_dawn) total_bounce_dawn,
SUM(total_bounce_breakfast) total_bounce_breakfast,
SUM(total_bounce_lunch) total_bounce_lunch,
SUM(total_bounce_snack) total_bounce_snack,
SUM(total_bounce_dinner) total_bounce_dinner,

SUM(total_click_dawn) total_click_dawn,
SUM(total_click_breakfast) total_click_breakfast,
SUM(total_click_lunch) total_click_lunch,
SUM(total_click_snack) total_click_snack,
SUM(total_click_dinner) total_click_dinner,

SUM(total_event) total_event,
SUM(total_send) total_send,
SUM(total_received) total_received,
SUM(total_bounce) total_bounce,
SUM(total_click) total_click

FROM push_user_day

GROUP BY 1, 2

In [0]:
df_push_user_dow_view = spark.table('push_user_dow_view')
df_push_user_dow_view.write.parquet(f'{root_dir}/enrich/push_user_dow', mode='overwrite')

In [0]:
df_push_user_dow = spark.read.parquet(f'{root_dir}/enrich/push_user_dow')
df_push_user_dow.createOrReplaceTempView('push_user_dow')
print((df_push_user_dow.count(), len(df_push_user_dow.columns)))

(193132, 32)

In [0]:
%sql
SELECT event_dow, total_click, total_bounce

FROM push_user_dow

WHERE external_user_id = 'cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4'

ORDER BY event_dow

event_dow,total_click,total_bounce
1,13,3
2,16,13
3,14,5
4,9,1
5,7,7
6,15,7
7,11,2


In [0]:
%sql
SELECT external_user_id, SUM(total_click) total_click, SUM(total_bounce) total_bounce
FROM push_user_dow
group by external_user_id
HAVING total_click > 0 AND total_bounce > 0

ORDER BY total_bounce desc, total_click desc

external_user_id,total_click,total_bounce
3c1b31e417179a571b363ba82868810816d7411ddeaa9dec1c52e3ccc020a1c7,8,62
929f2289a0001063f1137ab7673c58c54f026a83ce3e93daedb211d3f933176a,2,55
5c265dfbfc426ec6e4e234dc9cfa6be3d21fd699c5fcd6aaa3d64cb87521c38d,9,53
4e7f1915daee9e7e196c26e24f30dfd970b768f88a898baae31490fa8e314f9d,26,44
633ac940f578735a8a1fdfd0965229e4df7bd658fa8376fb73816f884ec8aa8c,5,40
cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,85,38
16c95bb5f6d87f7e0e4c68a92b9667eae76af96e6cdeed3cbb7be721ff05b479,8,36
0a03fd31166b66eac5bb1b4cf3fcf1679ab6b1aeb3770e91498f1764802e2d3c,5,31
aa9222da07814c231293d0ad8f25c12189c581127db1897944a0e5ed5583d50b,9,30
c944fc22dd0c120f48da1b9a5aa8f770a2e78be59a891ee229de1861bef029e2,49,29


# Session

In [0]:
df_sessions_visits = spark.read.parquet(f'{root_dir}/sessions_visits').repartition(2).cache()
print((df_sessions_visits.count(), len(df_sessions_visits.columns)))

df_orders = spark.read.parquet(f'{root_dir}/orders').repartition(2).cache()
print((df_orders.count(), len(df_orders.columns)))

(463620, 21)
(492138, 43)

In [0]:
df_sessions_visits.createOrReplaceTempView('session')
df_orders.createOrReplaceTempView('orders')

In [0]:
%sql
SELECT user_identifier, COUNT(*) total FROM session GROUP BY user_identifier ORDER BY total DESC

user_identifier,total
null,24916
fd9010344519e7e2d3ac43353ee23d9fa3c1e016add46814647e4f23cd06b1f5,193
485b1c9f4362e3126d647dff3898fe5005f7aef8da3288414b3de60b20a80cce,130
54dd771cff4d6f6d07ba3f252cfc64ca3edb3c5fdcc0ad03b83a45da569c5eb1,130
422b8ae235a1cd64746cd2138e543344155a69819f67478eb8ef033213b7df31,119
6be3e0c69908a7bd9add68111b31ba67882f28a0d9adb3ca192c447b644e679a,114
de9942aa571e3fe0df508824d84221f6486d890ba5f9a2660d7ef938b85361f7,113
2d8abc215aca4b055714f0baee81371b9333aa7d368118e695506353e58e246b,111
b3c70035-9816-4e59-9355-de8be76f3ac0,110
b63c278e637cb194f37a55948f4d4f04653ebf0563f320edc65a7105e40886c5,109


# Sumário diário de sessão do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW session_user_day_view AS
SELECT
  date_format(session_started_at_amsp, 'yyyy-MM-dd') session_date,
  dayofweek(session_started_at_amsp) session_dow,
  orders.customer_id user_identifier,
  SUM(IF(sum_event_open IS NULL, 0, sum_event_open)) sum_event_open,
  SUM(IF(sum_view_restaurant_screen IS NULL, 0, sum_view_restaurant_screen)) sum_view_restaurant_screen,
  SUM(IF(sum_view_dish_screen IS NULL, 0, sum_view_dish_screen)) sum_view_dish_screen,
  SUM(IF(sum_click_add_item IS NULL, 0, sum_click_add_item)) sum_click_add_item,
  SUM(IF(sum_view_checkout IS NULL, 0, sum_view_checkout)) sum_view_checkout,
  SUM(IF(sum_callback_purchase IS NULL, 0, sum_callback_purchase)) sum_callback_purchase,
  SUM(IF(order_session_quantity IS NULL, 0, order_session_quantity)) order_session_quantity

FROM session

INNER JOIN orders
ON orders.session_id = session.session_id

GROUP BY 1, 2, 3


In [0]:
df_session_user_day_view = spark.table('session_user_day_view')
df_session_user_day_view.write.parquet(f'{root_dir}/enrich/session_user_day', mode='overwrite')

In [0]:
df_session_user_day = spark.read.parquet(f'{root_dir}/enrich/session_user_day')
print((df_session_user_day.count(), len(df_session_user_day.columns)))

(435270, 10)

In [0]:
df_session_user_day.createOrReplaceTempView('session_user_day')

In [0]:
%sql
SELECT COUNT(*) total FROM session_user_day WHERE order_session_quantity = 0

total
1837


# Sumário mensal de sessão do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW session_user_month_view AS

SELECT
  date_format(to_timestamp(session_date), 'yyyy-MM-01') session_date,
  user_identifier,
  SUM(IF(sum_event_open IS NULL, 0, sum_event_open)) sum_event_open,
  SUM(IF(sum_view_restaurant_screen IS NULL, 0, sum_view_restaurant_screen)) sum_view_restaurant_screen,
  SUM(IF(sum_view_dish_screen IS NULL, 0, sum_view_dish_screen)) sum_view_dish_screen,
  SUM(IF(sum_click_add_item IS NULL, 0, sum_click_add_item)) sum_click_add_item,
  SUM(IF(sum_view_checkout IS NULL, 0, sum_view_checkout)) sum_view_checkout,
  SUM(IF(sum_callback_purchase IS NULL, 0, sum_callback_purchase)) sum_callback_purchase,
  SUM(IF(order_session_quantity IS NULL, 0, order_session_quantity)) order_session_quantity


FROM session_user_day

GROUP BY 1, 2

In [0]:
df_session_user_month_view = spark.table('session_user_month_view')
df_session_user_month_view.write.parquet(f'{root_dir}/enrich/session_user_month', mode='overwrite')

In [0]:
df_session_user_month = spark.read.parquet(f'{root_dir}/enrich/session_user_month')
print((df_session_user_month.count(), len(df_session_user_month.columns)))
df_session_user_month.createOrReplaceTempView('session_user_month')

(128357, 9)

In [0]:
%sql
SELECT *
FROM session_user_month
WHERE user_identifier = '6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5'

session_date,user_identifier,sum_event_open,sum_view_restaurant_screen,sum_view_dish_screen,sum_click_add_item,sum_view_checkout,sum_callback_purchase,order_session_quantity
2019-07-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,120.0,56.0,138.0,59.0,174.0,50.0,48.0
2019-11-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,13.0,3.0,13.0,2.0,3.0,2.0,2.0
2019-06-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,28.0,37.0,32.0,19.0,38.0,9.0,8.0
2019-08-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,12.0,6.0,12.0,9.0,12.0,6.0,8.0
2019-12-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,13.0,15.0,27.0,6.0,17.0,4.0,4.0
2019-09-01,6a24d4e7f0c107ef9172f3d7a5f4ccf81572fdf0f1fbc9c7fbb621da768f79d5,26.0,7.0,23.0,14.0,32.0,9.0,8.0


In [0]:
%sql
SELECT date_format(session_date, 'MM - MMM') session_date, SUM(sum_view_checkout) sum_view_checkout, sum(sum_callback_purchase) sum_callback_purchase
FROM session_user_month
group by session_date
ORDER BY session_date

session_date,sum_view_checkout,sum_callback_purchase
05 - May,76.0,13.0
06 - Jun,465348.0,128244.0
07 - Jul,332136.0,89147.0
08 - Aug,284400.0,86744.0
09 - Sep,251473.0,81116.0
10 - Oct,249356.0,82244.0
11 - Nov,272493.0,92505.0
12 - Dec,238537.0,83476.0


# Sumário semanal de sessão do cliente

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW session_user_dow_view AS

SELECT
  session_dow,
  user_identifier,
  SUM(IF(sum_event_open IS NULL, 0, sum_event_open)) sum_event_open,
  SUM(IF(sum_view_restaurant_screen IS NULL, 0, sum_view_restaurant_screen)) sum_view_restaurant_screen,
  SUM(IF(sum_view_dish_screen IS NULL, 0, sum_view_dish_screen)) sum_view_dish_screen,
  SUM(IF(sum_click_add_item IS NULL, 0, sum_click_add_item)) sum_click_add_item,
  SUM(IF(sum_view_checkout IS NULL, 0, sum_view_checkout)) sum_view_checkout,
  SUM(IF(sum_callback_purchase IS NULL, 0, sum_callback_purchase)) sum_callback_purchase,
  SUM(IF(order_session_quantity IS NULL, 0, order_session_quantity)) order_session_quantity


FROM session_user_day

GROUP BY 1, 2

In [0]:
df_session_user_dow_view = spark.table('session_user_dow_view')
df_session_user_dow_view.write.parquet(f'{root_dir}/enrich/session_user_dow', mode='overwrite')

In [0]:
df_session_user_dow = spark.read.parquet(f'{root_dir}/enrich/session_user_dow')
print((df_session_user_dow.count(), len(df_session_user_dow.columns)))

(129688, 9)

In [0]:
df_session_user_dow.createOrReplaceTempView('session_user_dow')

In [0]:
%sql
SELECT *

FROM session_user_dow

WHERE user_identifier = 'cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4'

ORDER BY session_dow

session_dow,user_identifier,sum_event_open,sum_view_restaurant_screen,sum_view_dish_screen,sum_click_add_item,sum_view_checkout,sum_callback_purchase,order_session_quantity
1,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,172.0,124.0,104.0,52.0,168.0,17.0,15.0
2,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,32.0,44.0,38.0,21.0,51.0,10.0,5.0
3,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,37.0,41.0,45.0,16.0,47.0,10.0,4.0
4,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,43.0,21.0,24.0,22.0,63.0,13.0,4.0
5,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,28.0,34.0,33.0,10.0,36.0,11.0,6.0
6,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,12.0,12.0,14.0,7.0,25.0,5.0,3.0
7,cdcd87191f9362a08ad708185a411ffeac30b29fcf52368c04a00d2ea25be7d4,3.0,43.0,15.0,7.0,25.0,1.0,1.0


# Sumário mensal de push e session

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW push_session_month_view AS

SELECT
if(session.session_date is null, push.event_date, session.session_date) customer_date,
if(session.user_identifier is null, push.external_user_id, session.user_identifier) customer_id,
*

FROM session_user_month session

FULL JOIN push_user_month push
ON session.user_identifier = push.external_user_id
AND session.session_date = push.event_date

In [0]:
df_push_session_month = spark.table('push_session_month_view')
df_push_session_month.write.parquet(f'{root_dir}/abt/push_session_month', mode='overwrite')

In [0]:
df_push_session_month = spark.read.parquet(
  f'{root_dir}/abt/push_session_month'
).repartition(2).cache()

df_push_session_month.createOrReplaceTempView('df_push_session_month')

In [0]:
df_push_session_jun = df_push_session_month.where('customer_date = "2019-06-01"')
df_push_session_jun.write.parquet(f'{root_dir}/abt/push_session_jun', mode='overwrite')

In [0]:
df_push_session_jul = df_push_session_month.where('customer_date = "2019-07-01"')
df_push_session_jul.write.parquet(f'{root_dir}/abt/push_session_jul', mode='overwrite')

In [0]:
df_push_session_aug = df_push_session_month.where('customer_date = "2019-08-01"')
df_push_session_aug.write.parquet(f'{root_dir}/abt/push_session_aug', mode='overwrite')

In [0]:
df_push_session_sep = df_push_session_month.where('customer_date = "2019-09-01"')
df_push_session_sep.write.parquet(f'{root_dir}/abt/push_session_sep', mode='overwrite')

In [0]:
df_push_session_oct = df_push_session_month.where('customer_date = "2019-10-01"')
df_push_session_oct.write.parquet(f'{root_dir}/abt/push_session_oct', mode='overwrite')

In [0]:
df_push_session_nov = df_push_session_month.where('customer_date = "2019-11-01"')
df_push_session_nov.write.parquet(f'{root_dir}/abt/push_session_nov', mode='overwrite')

In [0]:
df_push_session_dec = df_push_session_month.where('customer_date = "2019-12-01"')
df_push_session_dec.write.parquet(f'{root_dir}/abt/push_session_dec', mode='overwrite')